In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA as skPCA
from cuML import PCA as cumlPCA
import pygdf
import os

# Helper Functions

In [2]:
from timeit import default_timer

class Timer(object):
    def __init__(self):
        self._timer = default_timer
    
    def __enter__(self):
        self.start()
        return self

    def __exit__(self, *args):
        self.stop()

    def start(self):
        """Start the timer."""
        self.start = self._timer()

    def stop(self):
        """Stop the timer. Calculate the interval in seconds."""
        self.end = self._timer()
        self.interval = self.end - self.start

In [3]:
import gzip
def load_data(nrows, ncols, cached = 'data/mortgage.npy.gz'):
    if os.path.exists(cached):
        print('use mortgage data')
        with gzip.open(cached) as f:
            X = np.load(f)
        X = X[np.random.randint(0,X.shape[0]-1,nrows),:ncols]
    else:
        print('use random data')
        X = np.random.rand(nrows,ncols)
    df = pd.DataFrame({'fea%d'%i:X[:,i] for i in range(X.shape[1])})
    return df

In [4]:
def pd2pygdf(df):
    # convert pandas dataframe to pygdf dataframe
    if isinstance(df,np.ndarray):
        df = pd.DataFrame({'fea%d'%i:df[:,i] for i in range(df.shape[1])})
    pdf = pygdf.DataFrame()
    for c,column in enumerate(df):
        pdf[c] = df[column]
    return pdf

In [5]:
from sklearn.metrics import mean_squared_error
def array_equal(a,b,threshold=2e-3,with_sign=True):
    a = to_nparray(a)
    b = to_nparray(b)
    if with_sign == False:
        a,b = np.abs(a),np.abs(b)
    error = mean_squared_error(a,b)
    res = error<threshold
    return res

def to_nparray(x):
    if isinstance(x,np.ndarray) or isinstance(x,pd.DataFrame):
        return np.array(x)
    elif isinstance(x,np.float64):
        return np.array([x])
    elif isinstance(x,pygdf.DataFrame) or isinstance(x,pygdf.Series):
        return x.to_pandas().values
    return x    

# Run tests

In [6]:
%%time
nrows = 2**20
ncols = 400

X = load_data(nrows,ncols)
print('data',X.shape)

use mortgage data
data (1048576, 400)
CPU times: user 16.4 s, sys: 2.32 s, total: 18.7 s
Wall time: 18.7 s


In [7]:
n_components = 10
whiten = False
random_state = 42
svd_solver="full"

In [8]:
%%time
pca_sk = skPCA(n_components=n_components,svd_solver=svd_solver, 
            whiten=whiten, random_state=random_state)
result_sk = pca_sk.fit_transform(X)

CPU times: user 6min 7s, sys: 1min 37s, total: 7min 44s
Wall time: 21.3 s


In [9]:
%%time
X = pd2pygdf(X)

CPU times: user 8.03 s, sys: 2.4 s, total: 10.4 s
Wall time: 3.3 s


In [10]:
%%time
pca_cuml = cumlPCA(n_components=n_components,svd_solver=svd_solver, 
            whiten=whiten, random_state=random_state)
result_cuml = pca_cuml.fit_transform(X)

CPU times: user 3.37 s, sys: 496 ms, total: 3.87 s
Wall time: 3.89 s


In [11]:
for attr in ['singular_values_','components_','explained_variance_',
             'explained_variance_ratio_']:
    passed = array_equal(getattr(pca_sk,attr),getattr(pca_cuml,attr))
    message = 'compare pca: cuml vs sklearn {:>25} {}'.format(attr,'equal' if passed else 'NOT equal')
    print(message)

compare pca: cuml vs sklearn          singular_values_ equal
compare pca: cuml vs sklearn               components_ equal
compare pca: cuml vs sklearn       explained_variance_ equal
compare pca: cuml vs sklearn explained_variance_ratio_ equal


In [12]:
passed = array_equal(result_sk,result_cuml)
message = 'compare pca: cuml vs sklearn transformed results %s'%('equal'if passed else 'NOT equal')
print(message)

compare pca: cuml vs sklearn transformed results equal
